In [ ]:
# Устанавливаем необходимые пакеты
!pip install ultralytics
!pip install deep-sort-realtime

In [ ]:
# Подключаем необходимые библиотеки
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO
import numpy as np
import pandas as pd
from IPython.display import clear_output

In [ ]:
# Для deepsort используем значения по умолчанию
tracker = DeepSort()
# Используем предобученную модель
model = YOLO("yolov8n.pt")
# Загрузите свой видео файл в рабочий каталог
video = '/content/1.mp4'
# Здесь будем накапливать данные
people = np.empty((0, 5))
for result in model.predict(video, single_cls=True, classes=0):
  frame = result.orig_img
  result = result.cpu().boxes
  bbox = result.xywh.tolist()
  conf = result.conf.tolist()
  bbs = [(bbox[i], conf[i], 0) for i in range(len(bbox))]
  tracks = tracker.update_tracks(bbs, frame=frame)
  ids = np.array([int(track.track_id) for track in tracks])
  xyxys = np.array([track.to_ltrb() for track in tracks])
  if len(xyxys) > 0:
    people = np.vstack((people, np.hstack((xyxys, np.expand_dims(ids, axis=1)))))
clear_output()
df = pd.DataFrame(people, columns=['x1', 'y1', 'x2', 'y2', 'id'])
pd.options.mode.chained_assignment = None
# Будем формировать все возможные варианты
df.loc[:, 'last_x1'] = df.x1
df.loc[:, 'last_y1'] = df.y1
df.loc[:, 'last_x2'] = df.x2
df.loc[:, 'last_y2'] = df.y2
df.loc[:, 'min_x1'] = df.x1
df.loc[:, 'max_x1'] = df.x1
df.loc[:, 'min_y1'] = df.y1
df.loc[:, 'max_y1'] = df.y1
df.loc[:, 'min_x2'] = df.x2
df.loc[:, 'max_x2'] = df.x2
df.loc[:, 'min_y2'] = df.y2
df.loc[:, 'max_y2'] = df.y2
agg_func = {'x1': 'first', 'last_x1': 'last', 'min_x1': 'min', 'max_x1': 'max',
            'y1': 'first', 'last_y1': 'last', 'min_y1': 'min', 'max_y1': 'max',
            'x2': 'first', 'last_x2': 'last', 'min_x2': 'min', 'max_x2': 'max',
            'y2': 'first', 'last_y2': 'last', 'min_y2': 'min', 'max_y2': 'max'}
df1 = df.groupby('id').agg(agg_func)
df1.head(20)

,x1,last_x1,min_x1,max_x1,y1,last_y1,min_y1,max_y1,x2,last_x2,min_x2,max_x2,y2,last_y2,min_y2,max_y2
id,,,,,,,,,,,,,,,,
1.0,552.395935,552.420355,552.395935,552.420355,365.751709,365.711669,365.711669,365.751709,681.785828,681.782276,681.782276,681.785828,592.697144,592.605514,592.605514,592.697144
2.0,565.372986,511.010801,511.010801,590.125962,367.569702,323.942710,323.942710,397.027444,710.263977,633.607164,633.607164,717.103684,580.058105,538.255939,538.255939,624.590917
3.0,257.440002,1137.413073,257.440002,1137.413073,237.074936,493.958856,237.074936,493.958856,294.436218,1182.892099,294.436218,1182.892099,418.408081,557.199238,416.357454,643.308852
4.0,41.457546,41.459210,41.457546,41.459210,381.518646,381.522327,381.518646,381.522327,124.372635,124.364182,124.364182,124.372635,610.214844,610.190219,610.190219,610.214844
5.0,278.663025,275.885862,241.947068,279.240231,228.165817,411.223210,228.152841,411.223210,358.728516,421.091291,325.634097,421.091291,400.644409,917.151947,398.441174,917.151947
6.0,285.663635,609.250986,284.317160,609.250986,268.947357,466.842733,268.947357,466.842733,372.183899,723.730154,367.745325,723.730154,498.250580,644.904494,487.049332,644.904494
7.0,285.710266,285.668106,285.668106,285.710266,225.969421,226.044287,225.969421,226.044287,372.583435,372.615426,372.583435,372.615426,365.742493,365.939335,365.742493,365.939335
8.0,459.956604,460.901326,459.956604,460.901326,370.414307,370.160342,370.160342,370.414307,728.399719,728.778790,728.399719,728.778790,629.017212,628.192080,628.192080,629.017212
9.0,273.635559,268.491876,268.491876,275.684946,230.979828,368.044888,230.973591,368.044888,336.502747,405.484229,336.502747,405.484229,340.871063,613.908696,340.864800,613.908696


Слишком неустойчивая идентификация людей. Возможно изменение параметров улучшит ситуацию